In [1]:
import sys
import os

In [2]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K

In [3]:
K.clear_session()

In [6]:
data_training = './data/entrenamiento'
data_testing = './data/validacion'

In [7]:
epocas = 20
altura, longitud = 150, 150
batch_size = 32
pasos = 1000
pasos_validacion = 300
filtrosConv1 = 32
filtrosConv2 = 64
tamano_filtro1 = (3,3)
tamano_filtro2 = (2,2)
tamano_pool = (2,2)
clases = 42
lr = 0.0005

In [8]:
entrenamiento_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.3,
    zoom_range = 0.3,
    horizontal_flip = True
)

In [9]:
validacion_datagen = ImageDataGenerator(
    rescale = 1./255
)

In [10]:
imagen_entrenamiento = entrenamiento_datagen.flow_from_directory(
    data_training,
    target_size = (altura, longitud),
    batch_size = batch_size,
    class_mode = 'categorical'
)

FileNotFoundError: [Errno 2] No such file or directory: './data/entrenamiento'

In [31]:
imagen_validacion = validacion_datagen.flow_from_directory(
    data_testing,
    target_size = (altura, longitud),
    batch_size = batch_size,
    class_mode = 'categorical'
)

Found 84 images belonging to 42 classes.


In [32]:
cnn = Sequential()
cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding='same', input_shape = (altura, longitud, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding ="same", activation = 'relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

In [33]:
cnn.add(Flatten())
cnn.add(Dense(256, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(clases, activation='softmax'))

In [34]:
cnn.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=lr), metrics=['accuracy'])

In [35]:
cnn.fit_generator(
    imagen_entrenamiento,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=imagen_validacion,
    validation_steps=pasos_validacion
)

Epoch 1/20
1000/1000 [==============================] - 1466s 1s/step - loss: 3.5026 - acc: 0.1151 - val_loss: 3.6399 - val_acc: 0.0833
Epoch 2/20
 280/1000 [=======>......................] - ETA: 12:52 - loss: 3.2513 - acc: 0.1489

KeyboardInterrupt: 

In [ ]:
target_dir = './modelo/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
cnn.save('./modelo/modelo.h5')
cnn.save_weights('./modelo/pesos.h5')

# SEGUNDO INTENTO

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

In [2]:
DIRECTORIO = "./comida"
CATEGORIAS = ["alitas", "arroz", "bagel", "brownie", "croissant", "croquetas", "donas", "ensalada", "filete_de_res", 
              "hamburguesa", "hot_cakes", "hotdog",
             "huevos_revueltos", "lasagna", "muffin", "nachos", "nuggets_de_pollo", "omelet", "paella", "palomitas",
             "pan_tostado", "papas_a_la_francesa", "pastel", "pastel_de_queso", "pescado_frito", "pie_de_manzana",
             "pizza", "pollo_frito", "pollo_rostizado", "salchicha", "salmon", "sandwich", "spaghetti", "sushi", "tacos",
             "tiramisu", "tortilla", "vegetales_salteados", "waffle"]
salida = []

In [3]:
for categoria in CATEGORIAS:  #categorías
    path = os.path.join(DIRECTORIO,categoria)  # acceso a cada directorio de las categorías
    numero_categoria = CATEGORIAS.index(categoria)
    for img in tqdm(os.listdir(path)):  # Iterar cada imagen de perros y gatos
        try:
            img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # Convertir en un arreglo 
            nueva_img = cv2.resize(img_array, (70,70))
            salida.append([nueva_img, numero_categoria])
        except Exception as e:
            print(e)
            pass

100%|██████████| 102/102 [00:01<00:00, 55.95it/s]


In [4]:
import random
random.shuffle(salida)

In [5]:
X = []
y = []

In [6]:
for matriz, etiqueta in salida:
    X.append(matriz)
    y.append(etiqueta)

In [7]:
X = np.array(X).reshape(-1,70,70,1)

In [8]:
import pickle

In [9]:
pSalida = open("X.pickle","wb")
pickle.dump(X, pSalida)
pSalida.close()

pSalida = open("y.pickle","wb")
pickle.dump(y, pSalida)
pSalida.close()

pEntrada = open("X.pickle","rb")
X = pickle.load(pEntrada)

pEntrada = open("y.pickle","rb")
y = pickle.load(pEntrada)

In [10]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing import image

import pickle

In [11]:
X = X/255

In [12]:
modelo = Sequential()
modelo.add(Conv2D(64, (3,3), input_shape=X.shape[1:]))
modelo.add(Activation("relu"))
modelo.add(MaxPooling2D(pool_size=(2,2)))
modelo.add(Conv2D(64, (3,3)))
modelo.add(Activation("relu"))
modelo.add(MaxPooling2D(pool_size=(2,2)))
modelo.add(Conv2D(64, (3,3)))
modelo.add(Activation("relu"))
modelo.add(MaxPooling2D(pool_size=(2,2)))
modelo.add(Flatten())
modelo.add(Dense(64))
modelo.add(Dense(64))
modelo.add(Dense(39))
modelo.add(Activation("softmax"))

In [13]:
modelo.compile(optimizer="adam", loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [14]:
from keras.utils import to_categorical
y_cate = to_categorical(y)
print(len(y_cate))

Using TensorFlow backend.


4880


In [15]:
modelo.fit(X, y_cate, batch_size=32, epochs=20, validation_split=0.3)

Train on 3416 samples, validate on 1464 samples
Epoch 1/20
3416/3416 [==============================] - 33s 10ms/step - loss: 3.6524 - acc: 0.0448 - val_loss: 3.6298 - val_acc: 0.0410
Epoch 2/20
3416/3416 [==============================] - 30s 9ms/step - loss: 3.4668 - acc: 0.0893 - val_loss: 3.3697 - val_acc: 0.1189
Epoch 3/20
3416/3416 [==============================] - 34s 10ms/step - loss: 3.2182 - acc: 0.1440 - val_loss: 3.2395 - val_acc: 0.1373
Epoch 4/20
3416/3416 [==============================] - 40s 12ms/step - loss: 2.9684 - acc: 0.1988 - val_loss: 3.1993 - val_acc: 0.1564
Epoch 5/20
3416/3416 [==============================] - 37s 11ms/step - loss: 2.7764 - acc: 0.2430 - val_loss: 3.2165 - val_acc: 0.1612
Epoch 6/20
3416/3416 [==============================] - 39s 11ms/step - loss: 2.5344 - acc: 0.3056 - val_loss: 3.1083 - val_acc: 0.1776
Epoch 7/20
3416/3416 [==============================] - 37s 11ms/step - loss: 2.3363 - acc: 0.3522 - val_loss: 3.1172 - val_acc: 0.2056
E

In [16]:
def carga(rutaImg):
    imgSize = 70
    imgArray = cv2.imread(rutaImg, cv2.IMREAD_GRAYSCALE)
    nImgArray = cv2.resize(imgArray, (imgSize, imgSize))
    return nImgArray.reshape(-1, imgSize, imgSize, 1)

In [23]:
nombre = input("comida? ")
p = modelo.predict([carga(nombre)])
print(p)
print(CATEGORIAS[int(p[0][0])])
print(int(p[0][0]))

comida? /home/alfredo/Documentos/UltimoSemestre/SistemasInteligentes/Clasificador-de-Imagenes/hamburguesa.jpg
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
alitas
0
